# Error Correction in Reads

## Background Info

We can assemble a **genome** from a collection of **reads**. Even though genome sequencing was a breakthrough technology, sequencing machines that identify reads still produce errors a substantial percentage of the time, and these errors are unpredictable. It's difficult to determine if the machine has made an error, and where in the read the error has occurred. Therefore, error correction in reads is a vital first step in genome assembly.

* **Genome**: The collection of an organism's DNA taken from all its chromosomes.
* **Reads**: A substring of a genome used for genome assembly.

## Problem

As is the case with point mutations, the most common type of sequencing error occurs when a single nucleotide frmo a read is interpreted incorrectly.

**Given**: A collection of up to 1000 reads of equal length in FASTA format. Some of these reads were generated with a single-nucleotide error. For each read $s$ in the dataset, one of the following applies:

* $s$ was correctly sequenced and appears in the dataset at least twice (possibly as a reverse complement).
* $s$ is incorrect, it appears in the dataset exactly once, and its Hamming distance is 1 with respect to exactly one correct read in the dataset (or its reverse complement).

**Return**: A list of all corrections in the form "[old read] --> [new read]". (Each correction must be a single symbol subsitution).

## Solution Explaination

It was really helpful for me to read the definition of an incorrect read very closely to get the right answer. The problem says:
* $s$ is incorrect, it appears in the dataset exactly once, and its Hamming distance is 1 with respect to exactly one **correct read** in the dataset (or its reverse complement), <br> **<i>NOT** 
* $s$ is incorrect, it appears in the dataset exactly once, and its Hamming distance is 1 with respect to exactly **one read** in the dataset (or its reverse complement).
    
With that in mind, the steps to solving this problem is as follows:
1. Collect all the **correct** reads.
    1. Select all the reads that appear at least twice in the original FASTA file.
    2.1. Create reverse complements of all reads in the FASTA file.
    2.2. Select all reads that appear in both the FASTA file and in the reverse complement reads list.
2. Get the reverse complement reads that belong to all of the correct reads.
3. For each read in the FASTA file, check if there's a read that has a point mutation (Hamming Distance = 1) when compared to a correct read.

First, read in the FASTA file to create a dictionary of 'sequence ID: read', where sequence ID is the key and read is the corresponding value.

In [11]:
from collections import Counter # import Counter for later use

In [5]:
f = open('../Sample_Error_Correction_in_Reads.txt', 'r')
l = f.read().splitlines()
d = {}
for i in l:
    if i.startswith('>'):
        d[i[1:]] = ''
        key = i[1:]
    else:
        d[key] = i

d

{'Rosalind_52': 'TCATC',
 'Rosalind_44': 'TTCAT',
 'Rosalind_68': 'TCATC',
 'Rosalind_28': 'TGAAA',
 'Rosalind_95': 'GAGGA',
 'Rosalind_66': 'TTTCA',
 'Rosalind_33': 'ATCAA',
 'Rosalind_21': 'TTGAT',
 'Rosalind_18': 'TTTCC'}

Next, let's create the functions to calculate the Hamming Distance, and to create reverse complements of reads.

In [6]:
# calculates the Hamming Distance
def get_Hamming_Distance(read1, read2):
    dist = 0
    #read1 and read2 are of equal length as specified in the problem
    for i in range(len(read1)):
        if read1[i] != read2[i]:
            dist += 1
    return dist

# outputs the reverse complement of the input read
def get_rev_comp(read):
    rev_comp = ''
    for i in read[::-1]:
        if i == 'A':
            rev_comp += 'T'
        elif i == 'T':
            rev_comp += 'A'
        elif i == 'G':
            rev_comp += 'C'
        else:
            rev_comp += 'G'
    return rev_comp

Finally, the *get_error_corrections* function outputs a dictionary where the key is the old read that contains the point mutation and the value is the actual correct read.

In [15]:
def get_error_corrections(reads_dic):
    corrections_result = {}
    original_reads = [read for read in reads_dic.values()]
    correct_reads_dic = get_correct_reads(original_reads)
    correct_reads_lst = list(correct_reads_dic.keys()) + list(correct_reads_dic.values())

    # for each original read, check with all the correct reads to see if it's an incorrect read that has a single nucleotide mutation
    for read in correct_reads_lst:
        for original_read in original_reads:
            if is_Hamming_Distance_1(read, original_read):
                corrections_result[original_read] = read
    return corrections_result


# definition of a correct read = appears at least twice in the dataset, possibly as a reverse complement => get all the duplicates in the dataset
# input: a list that contains only the original reads
# output: a dictionary with a key: value pair of 'original read that are correct: its reverse complement'
def get_correct_reads(original_reads_lst):
    duplicates_in_original = {k: get_rev_comp(k) for k,v in Counter(original_reads_lst).items() if v > 1}
    rev_comp_lst = [get_rev_comp(read) for read in original_reads_lst]

    for read in original_reads_lst:
        for i in range(len(rev_comp_lst)):
            if read == rev_comp_lst[i]:
                duplicates_in_original[read] = original_reads_lst[i]

    return duplicates_in_original


# returns True if the Hamming Distance between read1 and read2 is 1
def is_Hamming_Distance_1(read1, read2):
    dist = get_Hamming_Distance(read1, read2)

    if dist == 1:
        return True
    return False



In [14]:
get_error_corrections(d)

{'TTTCC': 'TTTCA', 'TTCAT': 'TTGAT', 'GAGGA': 'GATGA'}

## Actual Dataset

In [17]:
f = open('../rosalind_corr.txt', 'r')
l = f.read().splitlines()
d = {}
for i in l:
    if i.startswith('>'):
        d[i[1:]] = ''
        key = i[1:]
    else:
        d[key] = i

In [22]:
print('The first 5 old read -> new read pairs from the output: ')
for old, new in list(get_error_corrections(d).items())[:5]:
    print(old + "->" + new)

The first 5 old read -> new read pairs from the output: 
GATAATGCGTTCAATGCTTGCATCTTCGTACAGCATCTCGGAGTATAGAA->GATAATGCGTTCAATGCTTACATCTTCGTACAGCATCTCGGAGTATAGAA
GATAATGCGTTCAATGCTTACATCTTCGTATAGCATCTCGGAGTATAGAA->GATAATGCGTTCAATGCTTACATCTTCGTACAGCATCTCGGAGTATAGAA
GATAATGCGTTCAATGCTTACATCTTCGTACAGCATCTCGGAGCATAGAA->GATAATGCGTTCAATGCTTACATCTTCGTACAGCATCTCGGAGTATAGAA
GTTTACTCTGGCAGCAGACAGGACCGAGATAATGCGTTCAATGCTTACAT->GTTTACTCTTGCAGCAGACAGGACCGAGATAATGCGTTCAATGCTTACAT
GTTTACTCTTGCAGCAGACAGGACCGAGATTATGCGTTCAATGCTTACAT->GTTTACTCTTGCAGCAGACAGGACCGAGATAATGCGTTCAATGCTTACAT


## Problem Solved!